# Select LSI features for the two class classification problem

In [1]:
from sklearn.datasets import fetch_20newsgroups
categories_Ct = ['comp.graphics', 'comp.os.ms-windows.misc' ,'comp.sys.ibm.pc.hardware','comp.sys.mac.hardware']
categories_Ra = ['rec.autos','rec.motorcycles','rec.sport.baseball','rec.sport.hockey']
twenty_train_Ct = fetch_20newsgroups(subset='train', categories=categories_Ct, shuffle=True, random_state=42)
twenty_test_Ct = fetch_20newsgroups(subset='test', categories=categories_Ct, shuffle=True, random_state=42)
twenty_train_Ra = fetch_20newsgroups(subset='train', categories=categories_Ra, shuffle=True, random_state=42)
twenty_test_Ra = fetch_20newsgroups(subset='test', categories=categories_Ra, shuffle=True, random_state=42)

In [3]:
import numpy as np
Ra_train_data =twenty_train_Ra.data[:2343]
Ra_train_target = map(lambda x: x+4, twenty_train_Ra.target[:2343])
Ra_test_target = map(lambda x: x+4, twenty_test_Ra.target)
twenty_train_all_X =twenty_train_Ct.data + Ra_train_data
twenty_train_all_y = np.array(twenty_train_Ct.target.tolist() + Ra_train_target)

twenty_test_all_X =twenty_test_Ct.data + twenty_test_Ra.data
twenty_test_all_y = np.array(twenty_test_Ct.target.tolist() + Ra_test_target)
twenty_train_all_y

array([3, 1, 2, ..., 7, 4, 4])

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer
import nltk
stemmer = PorterStemmer()

def XcountsafterStem(X, vocab,vocab_new):
    #vocab_new={}
    for i in vocab:
        word = stemmer.stem(i)
        if word in vocab_new:
            vocab_new[word].append(vocab[i])
        else:
            vocab_new[word]=[vocab[i]]
    term_num = len(vocab_new)
    doc_num = X.shape[0]
    X_new = np.array ([ [ 0 for i in range(term_num)] for j in range(doc_num) ])
    for i in range(doc_num):
        for j in range(len(vocab_new.keys())):
            idxs = vocab_new[vocab_new.keys()[j]]
            for idx in idxs:
                X_new[i][j] += X[i][idx]
    return X_new

count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(twenty_train_all_X)
X_test_counts = count_vect.transform(twenty_test_all_X) #for test data


# dic = count_vect.vocabulary_
# newDic={}
# X_train_counts = XcountsafterStem(X_train_counts.toarray(), dic,newDic)
# newDic={}
# X_test_counts = XcountsafterStem(X_test_counts.toarray(), dic,newDic)
# print X_train.counts.shape
# print X_test_counts.shape
# print len(newDic)

In [31]:
vocab_new={}
vocab = count_vect.vocabulary_
X = X_train_counts.toarray()
for i in vocab:
    word = stemmer.stem(i)
    if word in vocab_new:
        vocab_new[word].append(vocab[i])
    else:
        vocab_new[word]=[vocab[i]]
term_num = len(vocab_new)
doc_num = X.shape[0]
X_new = np.array ([ [ 0 for i in range(term_num)] for j in range(doc_num) ])


In [69]:
X = X_test_counts.toarray()
print X.shape
print term_num
print doc_num

(3150, 78726)
69497
4686


In [72]:
doc_num = 3150
X_new = np.array ([ [ 0 for i in range(term_num)] for j in range(doc_num) ])
keys = vocab_new.keys()
for i in range(doc_num):
    if i%100==0:
        print i
    for j in range(len(keys)):
        idxs = vocab_new[keys[j]]
        for idx in idxs:
            X_new[i][j]+=X[i][idx]
print X_new.shape

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
(3150, 69497)


In [66]:
type(X_new)

numpy.ndarray

In [73]:
print X_new.shape

(3150, 69497)


In [3]:
# from sklearn.feature_extraction.text import CountVectorizer
# from nltk import word_tokenize          
# from nltk.stem.porter import PorterStemmer
# import nltk
# stemmer = PorterStemmer()
# def stem_tokens(tokens, stemmer):
#     stemmed = []
#     for item in tokens:
#         stemmed.append(stemmer.stem(item))
#     return stemmed

# def tokenize(text):
#     tokens = nltk.word_tokenize(text)
#     stems = stem_tokens(tokens, stemmer)
#     return stems

# count_vect = CountVectorizer(tokenizer=tokenize, stop_words='english')
# X_train_counts = count_vect.fit_transform(twenty_train_all_X)
# X_test_counts = count_vect.transform(twenty_test_all_X) #for test data
# X_train_counts.shape
# X_test_counts.shape

(3150, 107143)

In [3]:
vect = CountVectorizer()
X_train = vect.fit_transform(twenty_train_all_X)
X_train.shape

NameError: name 'CountVectorizer' is not defined

In [63]:
X_train_counts = X_new
print X_train_counts.shape

(4686, 69497)


In [74]:
X_test_counts = X_new
print X_test_counts.shape

(3150, 69497)


In [75]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
print X_train_tfidf.shape
print X_test_tfidf.shape


(4686, 69497)
(3150, 69497)


In [76]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=50, random_state=42)
svd.fit(X_train_tfidf)
X_train_LSI = svd.transform(X_train_tfidf)
X_test_LSI = svd.transform(X_test_tfidf)
print X_train_LSI.shape
print X_test_LSI.shape
##################Finished Feature selectsion################

(4686, 50)
(3150, 50)


In [77]:
def f(x):
    if x>3:
        return 1
    else:
        return 0
y_train_LSI = np.array(map(f,twenty_train_all_y))
y_test_LSI = np.array(map(f,twenty_test_all_y))

In [78]:
from sklearn.svm import LinearSVC
svm_model = LinearSVC()
svm_model.fit(X_train_LSI,y_train_LSI)
y_test_predicted = svm_model.predict(X_test_LSI)
print 'accuracy: ', 1.0*sum(y_test_LSI==y_test_predicted) / len(y_test_LSI)

accuracy:  0.974603174603


In [79]:
###save X and y
np.save('./data/X_train_LSI', X_train_LSI)
np.save('./data/X_test_LSI', X_test_LSI)
np.save('./data/y_train_LSI', y_train_LSI)
np.save('./data/y_test_LSI', y_test_LSI)

# Featch data and select features for multi-class classification

In [5]:
from sklearn.datasets import fetch_20newsgroups
cat = ['comp.sys.ibm.pc.hardware','comp.sys.mac.hardware','misc.forsale','soc.religion.christian']
twenty_train_multi = fetch_20newsgroups(subset='train', categories=cat, shuffle=True, random_state=42)
twenty_test_multi = fetch_20newsgroups(subset='test', categories=cat, shuffle=True, random_state=42)

In [7]:
print len(twenty_train_multi.data)
print len(twenty_train_multi.filenames)
print len(twenty_train_multi.target)
print len(twenty_train_multi.target_names)
print twenty_train_multi.target_names
print twenty_train_multi.target[:50]

2352
2352
2352
4
['comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'misc.forsale', 'soc.religion.christian']
[3 2 0 2 3 0 2 3 0 1 2 1 1 2 0 2 1 3 1 0 0 3 0 1 1 1 1 2 3 3 0 2 3 0 2 0 1
 3 2 3 2 3 0 1 3 3 2 1 3 0]


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer
import nltk
stemmer = PorterStemmer()


count_vect_multi = CountVectorizer(stop_words='english')
X_train_counts_multi = count_vect_multi.fit_transform(twenty_train_multi.data)
X_test_counts_multi = count_vect_multi.transform(twenty_test_multi.data) #for test data
print X_train_counts_multi.shape
print X_test_counts_multi.shape

(2352, 31537)
(1565, 31537)


In [9]:
vocab_new={}
vocab = count_vect_multi.vocabulary_
for i in vocab:
    word = stemmer.stem(i)
    if word in vocab_new:
        vocab_new[word].append(vocab[i])
    else:
        vocab_new[word]=[vocab[i]]
term_num = len(vocab_new)
print term_num

25031


In [4]:
X = X_train_counts_multi.toarray()

In [11]:
import numpy as np
X = X_train_counts_multi.toarray()
doc_num = X.shape[0]
X_new = np.array ([ [ 0 for i in range(term_num)] for j in range(doc_num) ])
keys = vocab_new.keys()
for i in range(doc_num):
    if i%100==0:
        print i
    for j in range(len(keys)):
        idxs = vocab_new[keys[j]]
        for idx in idxs:
            X_new[i][j]+=X[i][idx]
print X_new.shape
X_train_counts_multi = X_new

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
(2352, 25031)


In [87]:
X = X_test_counts_multi.toarray()
doc_num = X.shape[0]
X_new = np.array ([ [ 0 for i in range(term_num)] for j in range(doc_num) ])
keys = vocab_new.keys()
for i in range(doc_num):
    if i%100==0:
        print i
    for j in range(len(keys)):
        idxs = vocab_new[keys[j]]
        for idx in idxs:
            X_new[i][j]+=X[i][idx]
print X_new.shape
X_test_counts_multi = X_new

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
(1565, 25031)


In [88]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf_multi = tfidf_transformer.fit_transform(X_train_counts_multi)
X_test_tfidf_multi = tfidf_transformer.transform(X_test_counts_multi)
print X_train_tfidf_multi.shape
print X_test_tfidf_multi.shape

(2352, 25031)
(1565, 25031)


In [89]:
from sklearn.decomposition import TruncatedSVD
svd_multi = TruncatedSVD(n_components=50, random_state=42)
svd_multi.fit(X_train_tfidf_multi)
X_train_LSI_multi = svd_multi.transform(X_train_tfidf_multi)
X_test_LSI_multi = svd_multi.transform(X_test_tfidf_multi)
print X_train_LSI_multi.shape
print X_test_LSI_multi.shape

(2352, 50)
(1565, 50)


In [90]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
label_test = OneVsOneClassifier(LinearSVC(random_state=0)).fit(X_train_LSI_multi, twenty_train_multi.target).predict(X_test_LSI_multi)
print 'accuracy: ', 1.0*sum(label_test==twenty_test_multi.target) / twenty_test_multi.target.shape[0]

accuracy:  0.877955271565


In [93]:
###save X and y for multi-class classification
np.save('./data/X_train_LSI_multi', X_train_LSI_multi)
np.save('./data/X_test_LSI_multi', X_test_LSI_multi)
np.save('./data/y_train_LSI_multi', twenty_train_multi.target)
np.save('./data/y_test_LSI_multi', twenty_test_multi.target)